# Experiments page for the programming Language 'Phi'
It is a project aiming to create a math friendly programming language.

In [33]:
# Imports
import re
import json

In [34]:
# This is the line which will be used throughout the notebook
TEST_LINE = "10 + b"

In [35]:
class Error(Exception):
    """Base class for exceptions in this module."""
    def __init__(self, msg):
        self.msg = msg
        super().__init__(self.msg)


class ParseError(Error):
    """Exception raised for errors in the input.
    Attributes:
        msg  -- explanation of the error
    """

    def __init__(self, msg):
        self.msg = 'PARSING ERROR : ' + msg
        super().__init__(self.msg)

# Tokens

In [36]:
# Lexer
r_ID = r'[a-zA-Z_][a-zA-Z0-9_]*'
r_Num = r'(\d+)\.?(\d+)*'
r_Assign = r'='
r_Comma = r'\,'
r_Plus = r'\+'
r_Minus = r'-'
r_Mult = r'\*'
r_Div = r'/'
r_Dot = r'\.'
r_Caret = r'\^'
r_lParen = r'\('
r_rParen = r'\)'
r_lBrace = r'\{'
r_rBrace = r'\}'

# Keywords
keywords = {
    'func': 'FUNC',
    'print': 'PRINT',
    'show': 'SHWTBL',
    'return': 'RETURN',
    'plot': 'PLT',
}

# Token Class


class Token:
    def __init__(self, type, value=None):
        self.type = type
        self.value = value

    def __str__(self):
        return f"< Token {self.type} :: '{self.value}' >" if self.value else f'< Token {self.type} >'

    def __repr__(self):
        return self.__str__()


class TupleToken(Token):
    def __init__(self):
        super().__init__(type='TUPLE')
        self.variables = []
        self.values = []

    def add(self, token):
        if isinstance(token, Token):
            self.variables.append(str(token))
            self.values = self.variables

    def __str__(self):
        return f"< Tuple :: {' ,'.join(self.variables)} >"

# Lexer

In [37]:
# Lexer Class

class Lexer:
    def __init__(self, line) -> None:
        self.line = line
        self.pos = -1
        self.char = None
        self.token_corpus = []
        self.advance()

    def advance(self):
        if self.pos < len(self.line) - 1:
            self.pos += 1
            self.char = self.line[self.pos]

        else:
            self.char = None
            self.token_corpus.append(Token('EOL'))

    def peek(self):
        return self.line[self.pos + 1] if self.pos < len(self.line) - 1 else 'EOL'

    def assert_char(self, char, tokentype):
        if re.fullmatch(tokentype, char):
            return True
        else:
            return False

    def make_number(self):
        num = self.char
        next_char = self.peek()
        while self.assert_char(num+next_char, r_Num) and next_char != 'EOL':
            num += next_char
            self.advance()
            next_char = self.peek()

        return Token('NUMBER', num)

    def make_id(self):
        if not self.assert_char(self.char, r_ID):
            return
        id = self.char
        next_char = self.peek()
        while self.assert_char(id+next_char, r_ID) and next_char != 'EOL':
            id += next_char
            self.advance()
            next_char = self.peek()

        if id in keywords:
            return Token(keywords[id])
        else:
            return Token('ID', id)

    def get_tokens(self):
        while self.char != None:
            if self.assert_char(self.char, r_Num):
                self.token_corpus.append(self.make_number())
            elif self.assert_char(self.char, r_ID):
                self.token_corpus.append(self.make_id())
            elif self.assert_char(self.char, r_lParen):
                self.token_corpus.append(Token('LPAREN'))
            elif self.assert_char(self.char, r_rParen):
                self.token_corpus.append(Token('RPAREN'))
            elif self.assert_char(self.char, r_Comma):
                self.token_corpus.append(Token('COMMA'))
            elif self.assert_char(self.char, r_Plus):
                self.token_corpus.append(Token('PLUS'))
            elif self.assert_char(self.char, r_Minus):
                self.token_corpus.append(Token('MINUS'))
            elif self.assert_char(self.char, r_Mult):
                self.token_corpus.append(Token('MULT'))
            elif self.assert_char(self.char, r_Div):
                self.token_corpus.append(Token('DIV'))
            elif self.assert_char(self.char, r_Dot):
                self.token_corpus.append(Token('DOT'))
            elif self.assert_char(self.char, r_Assign):
                self.token_corpus.append(Token('ASSIGN'))
            elif self.assert_char(self.char, r_Caret):
                self.token_corpus.append(Token('CARET'))
            elif self.assert_char(self.char, r_lBrace):
                self.token_corpus.append(Token('LBRACE'))
            elif self.assert_char(self.char, r_rBrace):
                self.token_corpus.append(Token('RBRACE'))

            elif self.char == ' ':
                pass
            elif self.char == '\n':
                self.token_corpus.append(Token('EOL'))
            else:
                raise Exception(f'Invalid Character: {self.char}')
            self.advance()
        return self.token_corpus

In [38]:

l = Lexer(TEST_LINE)
l.get_tokens()

[< Token NUMBER :: '10' >, < Token PLUS >, < Token ID :: 'b' >, < Token EOL >]

# Parser - Expreiment 01 : Expression Parser
Experiments for parsing an expression

In [39]:
tokens = l.get_tokens()
print(tokens)

[< Token NUMBER :: '10' >, < Token PLUS >, < Token ID :: 'b' >, < Token EOL >]


In [40]:
class BinOpNode:
    def __init__(self, left, operator, right):
        self.type = "BINOP"
        self.left = left
        self.operator = operator
        self.right = right

    def __str__(self) -> str:
        return f"({self.left} {self.operator} {self.right})"

    def __repr__(self) -> str:
        return self.__str__()


class FactorNode:
    def __init__(self, value, sign='+'):
        self.type = "FACTOR"
        self.value = value if sign == '+' else '-' + value

    def __str__(self) -> str:
        return f"{self.value.value}"

    def __repr__(self) -> str:
        return self.__str__()


class ExpressionNode:
    def __init__(self, expression, type_hint=None):
        self.type = "EXPRESSION"
        self.expression = expression
        self.type_hint = type_hint # Used for a special case when the expression is a single number
        self.value = f"Expression {self.expression}"
        
    def __str__(self) -> str:
        return f"< Expression {self.expression}>"

    def __repr__(self) -> str:
        return self.__str__()


class DeclarationNode:
    def __init__(self, func: Token, args: TupleToken):
        self.type = "FUNCTION"
        self.function_name = func.value
        self.args = args
        self.value = f"Function {self.function_name} args {self.args.variables}"

    def __str__(self):
        return f"< Function {self.function_name} args {self.args.variables}>"

    def __repr__(self):
        return self.__str__()

In [41]:
class ExpressionParser:
    def __init__(self, tokens):
        self.master_tokens = tokens
        self.tokens = tokens
        self.current_token: Token = None
        self.index = -1
        self.startindex, self.endindex = 0, 0
        self.advance()

    def advance(self):
        while self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
            self.index += 1
            if self.current_token.type != 'EOL':
                break

    def peek(self):
        return self.tokens[self.index+1] if self.index+1 < len(self.tokens) else None

    def return_slice(self):
        return self.startindex, self.endindex

    def parse(self):
        self.startindex = self.index - 1
        parsed = self.parse_expression()
        self.endindex = self.index - 1

        if getattr(parsed, 'type', None) is not None and parsed.type == "FACTOR":
            if parsed.value.type in ('ID', 'FUNCTION'):
                parsed = parsed.value
                return parsed
            if parsed.value.type == 'NUMBER':
                parsed = parsed.value.value
                return ExpressionNode(parsed, type_hint='NUM')
            
        elif not parsed:
            return parsed
        return ExpressionNode(parsed)

    def parse_expression(self):
        left = self.parse_term()
        while self.current_token.type in ('PLUS', 'MINUS'):
            op = self.current_token.type
            self.advance()
            right = self.parse_term()
            left = BinOpNode(left, op, right)
        return left

    def parse_term(self):
        left = self.parse_radical()
        while self.current_token.type in ('MULT', 'DIV', 'DOT'):
            op = self.current_token.type if self.current_token.type not in (
                'DOT') else 'MULT'
            self.advance()
            right = self.parse_radical()
            left = BinOpNode(left, op, right)

        return left

    def parse_radical(self):
        left = self.parse_factor()
        while self.current_token.type == 'CARET':
            op = self.current_token.type
            self.advance()
            right = self.parse_factor()
            left = BinOpNode(left, op, right)

        return left

    def parse_factor(self):
        if self.current_token.type == 'NUMBER':

            value = self.current_token
            self.advance()
            return FactorNode(value)

        elif self.current_token.type == 'ID':
            value = self.current_token
            self.advance()
            return FactorNode(value)

        elif self.current_token.type == 'FUNCTION':
            value = self.current_token
            self.advance()
            return FactorNode(value)

        elif self.current_token.type == 'MINUS':  # Unary negation handling
            self.advance()
            right = self.parse_factor()
            return FactorNode(right, sign='-')

        # Handle opening parenthesis "("
        elif self.current_token.type == 'LPAREN':
            self.advance()
            expr = self.parse_expression()
            if self.current_token.type == 'COMMA':
                pass  # Ensure closing parenthesis ")"
            elif self.current_token.type == 'RPAREN':  # Ensure closing parenthesis ")"
                self.advance()
                return expr
            else:
                raise SyntaxError("Missing closing parenthesis")

In [42]:
class ExpressionParserWrapper:
    def __init__(self, tokens):
        self.index = -1
        self.tokens = tokens
        self.current_token: Token = None

        self.advance()

    def advance(self):
        while self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
            self.index += 1
            if self.current_token.type != "EOL":
                break

    def return_tokens(self):
        return self.tokens

    def parse(self):
        while self.current_token.type != "EOL":
            print(self.current_token)
            ep = ExpressionParser(self.tokens[self.index-1:])
            pr = ep.parse()
            sl1, sl0 = [i + self.index - 1 for i in ep.return_slice()]
            if pr:
                self.tokens[sl1:sl0] = [pr]
                self.index = sl1 + 1
            self.advance()
        return self.return_tokens()

In [43]:
p = ExpressionParser(tokens)
print(tokens)
print(p.parse())
sl = p.return_slice()
print(sl)
print(tokens[sl[0]:sl[1]])

[< Token NUMBER :: '10' >, < Token PLUS >, < Token ID :: 'b' >, < Token EOL >]
< Expression (10 PLUS b)>
(0, 3)
[< Token NUMBER :: '10' >, < Token PLUS >, < Token ID :: 'b' >]


## Parser - Experiment 02 : Tuple Parser
Experiments for parsing a tuple

In [44]:
class TupleParser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = tokens[0]
        self.tuple = TupleToken
        self.index = -1
        self.item_corpus = []
        self.advance()

    def advance(self):
        self.index += 1
        if self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
        else:
            self.current_token = Token('EOL')

    def couplet(self, items, stindex):
        return items[stindex], items[stindex+1]

    def assert_type(self, token, type, enforceable=False, alt_type=None):
        if token.type not in (type, alt_type):
            if enforceable:
                raise Exception('Expected token type ' +
                                type + ' but got ' + token.type)
            else:
                return False
        else:
            return True

    def parse_item(self, items):
        tupletok = TupleToken()
        index = 0
        # check if the list is empty
        if len(items) == 0:
            return tupletok
        # check if the list has only one item
        elif len(items) == 1:
            if self.assert_type(items[0], 'ID') or self.assert_type(items[0], 'NUMBER'):
                tupletok.add(items[0])
                return tupletok

        # check if the list has the format of a tuple
        id, sep = self.couplet(items, 0)
        if (self.assert_type(id, 'ID') or self.assert_type(id, 'NUMBER')) and self.assert_type(sep, 'COMMA'):
            tupletok.add(id)
            index += 1
            while index < len(items):
                sep, id = self.couplet(items, index)
                self.assert_type(sep, 'COMMA', enforceable=True)
                self.assert_type(id, 'ID', enforceable=True, alt_type='NUMBER')
                tupletok.add(id)
                index += 2

            return tupletok
        else:
            return None

    def replace(self, st, end, new_item):
        self.tokens[st:end] = [new_item]
        self.index = st

    def parse(self):
        t_items = []
        while self.index < len(self.tokens):
            if self.current_token.type == 'LPAREN':
                st = self.index
                while self.current_token.type != 'RPAREN' and self.current_token.type != 'EOL':
                    t_items.append(self.current_token)
                    self.advance()
                if self.tokens[self.index].type == 'RPAREN':
                    t_items.pop(0)
                    end = self.index + 1
                    self.replace(st, end, self.parse_item(t_items))

                    t_items = []

                elif self.tokens[self.index].type == 'EOL':
                    print('No closing parenthesis')

            self.advance()

        self.index = -1
        self.parse_declarations()
        return self.tokens

    def parse_declarations(self):
        while self.index < len(self.tokens):
            if self.current_token and self.current_token.type == 'ID' and self.index + 1 < len(self.tokens) and self.tokens[self.index+1].type == 'TUPLE':
                print('Declaration')
                self.replace(self.index, self.index+2,
                             DeclarationNode(self.current_token, self.tokens[self.index+1]))

            self.advance()

In [45]:
l = "plot f(x)"
le = Lexer(l)
l = le.get_tokens()
# print(l)
tp = TupleParser(l)
l = tp.parse()
# print(l)
epw = ExpressionParserWrapper(l)
tok = epw.parse()
print(tok)

Declaration
< Token PLT >
< Function f args ["< Token ID :: 'x' >"]>
[< Token PLT >, < Function f args ["< Token ID :: 'x' >"]>, < Token EOL >]


In [46]:
class LineNode:
    def __init__(self, tokens:list, specid:str, primarykeyword:str, grammar:dict):
        self.type = "LINE"
        self.tokens = tokens
        self.grammar = grammar
        self.mastergrammar = specid
        self.primarykeyword = primarykeyword
        self.function = self.grammar[self.primarykeyword]['function']        
        
    
    def __str__(self) -> str:
        return f"[LINE grammar {self.mastergrammar[0]}:{self.mastergrammar[0:]} <Contents{self.tokens}>]"

    def __repr__(self) -> str:
        return self.__str__()

In [47]:


class MasterParser:
    def __init__(self, tokens, grammar):
        self.tokens = tokens
        self.grammar_raw = grammar
        self.master_lexeme = None
        self.applicable_rules = []
        self.absolute_rule = None
        self.index = -1
        self.current_token: Token = None
        self.enforce_grammar = False
        self.advance()

    def reset_index(self):
        self.index = -1
        self.current_token: Token = None
        self.advance()

    def advance(self):
        while self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
            self.index += 1
            if self.current_token.type != "EOL":
                break

    def get_master_rule(self):
        first_token_type = self.current_token.type
        self.primary_keyword = first_token_type
        if first_token_type in self.grammar_raw:
            self.master_lexeme = first_token_type
            self.applicable_rules = [self.grammar_raw[first_token_type]['syntax'][i]['syntax'].split()
                                     for i in self.grammar_raw[first_token_type]['syntax']
                                     ]
            self.rule_ids = [self.grammar_raw[first_token_type]['syntax'][i]['spec_id']
                              for i in self.grammar_raw[first_token_type]['syntax']
                              ]

    def get_item(self, index, list):
        if index < len(list):
            return list[index]
        else:
            return None

    def grammer_match(self):
        print(self.applicable_rules)
        for i in range(len(self.tokens)):
            self.enforce_grammar = True if len(
                self.applicable_rules) == 1 else False
            rules,rules_id = [],[]
            
            if self.enforce_grammar:
                if self.get_item(i, self.applicable_rules[0]) != None:
                    if self.get_item(i, self.applicable_rules[0]) == self.tokens[i].type or self.get_item(i, self.applicable_rules[0]) == getattr(self.tokens[i], 'type_hint', None):
                        pass
                    else:
                        if self.get_item(i, self.applicable_rules[0]) == 'EOL':
                            raise ParseError(f'Unexpected {self.tokens[i].type} \'{self.tokens[i].value}\'')
                        raise ParseError(f'Expected {self.get_item(i,self.applicable_rules[0])} but recieved {self.tokens[i].type} \'{self.tokens[i].value}\'')
                else:
                    raise ParseError(f'Unexpected {self.tokens[i].type} \'{self.tokens[i].value}\'')

            else:
                for j in range(len(self.applicable_rules)):
                    rule = self.applicable_rules[j]
                    id = self.rule_ids[j]
                    if self.get_item(i, rule) == self.tokens[i].type or self.get_item(i, rule) == getattr(self.tokens[i], 'type_hint', None):
                        rules.append(rule)
                        rules_id.append(id)
                        print(self.tokens[i].type, 'matched', rule)
                        print(self.applicable_rules)
                self.applicable_rules = rules
                self.rule_ids = rules_id
                
        if len(self.applicable_rules) != 1:
            raise ParseError(f'Syntax Error')
        elif len(self.applicable_rules) == 1:
            self.absolute_rule = self.applicable_rules[0]
            self.absolute_rule_id = self.rule_ids[0]
            print('Absolute Rule', self.absolute_rule)
            print('Absolute Rule ID', self.absolute_rule_id)
    
    
    
    def parse(self):
        self.get_master_rule()
        self.grammer_match()
        line = LineNode(self.tokens,self.absolute_rule_id,self.primary_keyword,self.grammar_raw)
        return line


In [48]:
with open('grammar.json', 'r') as f:
    grammar = json.load(f)

parser = MasterParser(tok, grammar)
parser.parse()

[['PLT', 'FUNCTION', 'EOL']]
Absolute Rule ['PLT', 'FUNCTION', 'EOL']
Absolute Rule ID G01


[LINE grammar G:G01 <Contents[< Token PLT >, < Function f args ["< Token ID :: 'x' >"]>, < Token EOL >]>]

In [49]:
print(grammar['PLT']['function'])

Plot
